# Tarea 2: Conteo Aproximado con MCMC
## Modelos Hard-Core y q-coloraciones

**Profesor:** Freddy Hernández-Romero

**Integrantes del grupo:**
- [Nombre 1]
- [Nombre 2]
- [Nombre 3]

**Fecha de entrega:** [Fecha]

In [ ]:
# Importaciones necesarias
import sys
import os

# Agregar el directorio padre al path
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Importar módulos desde src
from src.mcmc_counting import (
    LatticeGraph, 
    QColoringMCMC, 
    HardCoreMCMC, 
    MCMCConfig,
    exact_q_colorings_small, 
    exact_hardcore_small
)

print("Librerías importadas correctamente")

## 1. Aproximación del Número de q-Coloraciones

### 1.a) Experimentos con el Algoritmo de Conteo Aproximado

Implementaremos el algoritmo basado en el **Teorema 9.1** que establece que para grafos con grado máximo $d$ y $q > 2d^*$, existe un esquema de aproximación polinomial aleatorizado.

In [ ]:
def run_q_coloring_experiments(K_values, q_values, epsilons):
    """
    Ejecuta experimentos de conteo aproximado para q-coloraciones.
    
    Args:
        K_values: Lista de tamaños de lattice
        q_values: Lista de números de colores
        epsilons: Lista de valores de precisión
    
    Returns:
        DataFrame con resultados
    """
    results = []
    
    total_experiments = len(K_values) * len(q_values) * len(epsilons)
    with tqdm(total=total_experiments, desc="Experimentos q-coloraciones") as pbar:
        
        for K in K_values:
            lattice = LatticeGraph(K)
            max_degree = lattice.max_degree()
            
            for q in q_values:
                if q <= 2 * max_degree:
                    pbar.update(len(epsilons))
                    continue
                
                for epsilon in epsilons:
                    config = MCMCConfig(epsilon=epsilon)
                    mcmc = QColoringMCMC(lattice, q, config)
                    
                    estimate, stats = mcmc.count_approximate(verbose=False)
                    
                    results.append({
                        'K': K,
                        'q': q,
                        'epsilon': epsilon,
                        'estimate': estimate,
                        'num_simulations': stats['num_simulations'],
                        'mixing_time': stats['mixing_time'],
                        'elapsed_time': stats['elapsed_time'],
                        'max_degree': max_degree,
                        'condition_met': q > 2 * max_degree
                    })
                    
                    pbar.update(1)
    
    return pd.DataFrame(results)

In [ ]:
# Parámetros del experimento
K_values = [3, 4, 5, 6, 8, 10, 12, 15, 18, 20]
q_values = [2, 3, 4, 5, 6, 8, 10, 12, 15]
epsilons = [0.5, 0.2, 0.1, 0.05]

print("Configuración de experimentos:")
print(f"  Tamaños de lattice (K): {K_values}")
print(f"  Número de colores (q): {q_values}")
print(f"  Valores de epsilon: {epsilons}")

# Ejecutar experimentos
df_q_coloring = run_q_coloring_experiments(K_values[:5], q_values[:5], epsilons[:2])

In [ ]:
# Mostrar resumen de resultados
print("\n=== Resumen de Resultados de q-Coloraciones ===")
print("\nPrimeras filas de resultados:")
print(df_q_coloring.head(10))

# Análisis por epsilon
print("\n=== Análisis por Precisión (epsilon) ===")
for eps in df_q_coloring['epsilon'].unique():
    df_eps = df_q_coloring[df_q_coloring['epsilon'] == eps]
    print(f"\nEpsilon = {eps}:")
    print(f"  - Simulaciones promedio: {df_eps['num_simulations'].mean():.0f}")
    print(f"  - Tiempo de mezcla promedio: {df_eps['mixing_time'].mean():.0f}")
    print(f"  - Tiempo ejecución promedio: {df_eps['elapsed_time'].mean():.2f} segundos")

### Visualización de Resultados

In [ ]:
# Visualización 1: Estimaciones vs tamaño del lattice
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Para diferentes valores de q
for q in df_q_coloring['q'].unique()[:3]:
    df_q = df_q_coloring[(df_q_coloring['q'] == q) & (df_q_coloring['epsilon'] == epsilons[0])]
    if not df_q.empty:
        axes[0].semilogy(df_q['K'], df_q['estimate'], 'o-', label=f'q={q}')

axes[0].set_xlabel('Tamaño del Lattice (K)')
axes[0].set_ylabel('Número de q-coloraciones (log scale)')
axes[0].set_title(f'Estimaciones de q-coloraciones (ε={epsilons[0]})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Tiempo de ejecución vs tamaño
for eps in epsilons[:2]:
    df_eps = df_q_coloring[(df_q_coloring['epsilon'] == eps) & (df_q_coloring['q'] == 5)]
    if not df_eps.empty:
        axes[1].plot(df_eps['K'], df_eps['elapsed_time'], 'o-', label=f'ε={eps}')

axes[1].set_xlabel('Tamaño del Lattice (K)')
axes[1].set_ylabel('Tiempo de Ejecución (segundos)')
axes[1].set_title('Tiempo de Ejecución vs Tamaño del Lattice (q=5)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.b) Comparación con Conteo Exacto

Para lattices pequeños, comparamos nuestras aproximaciones con el conteo exacto.

In [ ]:
def compare_with_exact():
    """
    Compara aproximaciones con valores exactos para lattices pequeños.
    """
    K_small = [2, 3]
    q_small = [3, 4, 5]
    epsilon = 0.1
    
    results = []
    
    print("\n=== Comparación con Conteo Exacto ===")
    print("K\tq\tExacto\t\tAproximado\tError Relativo (%)")
    print("-" * 60)
    
    for K in K_small:
        lattice = LatticeGraph(K)
        
        for q in q_small:
            exact_count = exact_q_colorings_small(K, q)
            
            config = MCMCConfig(epsilon=epsilon, num_samples=500)
            mcmc = QColoringMCMC(lattice, q, config)
            approx_count, _ = mcmc.count_approximate(verbose=False)
            
            error = abs(approx_count - exact_count) / exact_count * 100
            
            print(f"{K}\t{q}\t{exact_count:,}\t\t{approx_count:.0f}\t\t{error:.2f}%")
            
            results.append({
                'K': K,
                'q': q,
                'exact': exact_count,
                'approximate': approx_count,
                'relative_error': error
            })
    
    return pd.DataFrame(results)

df_comparison = compare_with_exact()

In [ ]:
# Visualización de la comparación
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.scatter(df_comparison['exact'], df_comparison['approximate'], s=100, alpha=0.6)

max_val = max(df_comparison['exact'].max(), df_comparison['approximate'].max())
ax.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='y=x (perfecto)')

for _, row in df_comparison.iterrows():
    ax.annotate(f"K={row['K']}, q={row['q']}", 
                (row['exact'], row['approximate']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

ax.set_xlabel('Conteo Exacto')
ax.set_ylabel('Conteo Aproximado')
ax.set_title('Comparación: Conteo Exacto vs Aproximado\nq-coloraciones')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n=== Estadísticas de Error ===")
print(f"Error relativo promedio: {df_comparison['relative_error'].mean():.2f}%")
print(f"Error relativo máximo: {df_comparison['relative_error'].max():.2f}%")
print(f"Error relativo mínimo: {df_comparison['relative_error'].min():.2f}%")

## 2. Aproximación del Número de Configuraciones del Modelo Hard-Core

El modelo Hard-Core representa configuraciones donde dos vértices adyacentes no pueden estar ocupados simultáneamente.

In [ ]:
def run_hardcore_experiments(K_values, epsilons):
    """
    Ejecuta experimentos de conteo aproximado para el modelo Hard-Core.
    
    Args:
        K_values: Lista de tamaños de lattice
        epsilons: Lista de valores de precisión
    
    Returns:
        DataFrame con resultados
    """
    results = []
    
    total_experiments = len(K_values) * len(epsilons)
    with tqdm(total=total_experiments, desc="Experimentos Hard-Core") as pbar:
        
        for K in K_values:
            lattice = LatticeGraph(K)
            
            for epsilon in epsilons:
                config = MCMCConfig(epsilon=epsilon)
                mcmc = HardCoreMCMC(lattice, config)
                
                estimate, stats = mcmc.count_approximate(verbose=False)
                
                results.append({
                    'K': K,
                    'epsilon': epsilon,
                    'estimate': estimate,
                    'num_simulations': stats['num_simulations'],
                    'mixing_time': stats['mixing_time'],
                    'elapsed_time': stats['elapsed_time'],
                    'n_vertices': K * K
                })
                
                pbar.update(1)
    
    return pd.DataFrame(results)

In [ ]:
# Parámetros del experimento Hard-Core
K_values_hc = [3, 4, 5, 6, 8, 10, 12, 15, 18, 20]
epsilons_hc = [0.5, 0.2, 0.1]

print("Configuración de experimentos Hard-Core:")
print(f"  Tamaños de lattice (K): {K_values_hc}")
print(f"  Valores de epsilon: {epsilons_hc}")

# Ejecutar experimentos
df_hardcore = run_hardcore_experiments(K_values_hc[:6], epsilons_hc)

In [ ]:
# Mostrar resumen de resultados Hard-Core
print("\n=== Resumen de Resultados del Modelo Hard-Core ===")
print("\nPrimeras filas de resultados:")
print(df_hardcore.head(10))

# Análisis por epsilon
print("\n=== Análisis por Precisión (epsilon) ===")
for eps in df_hardcore['epsilon'].unique():
    df_eps = df_hardcore[df_hardcore['epsilon'] == eps]
    print(f"\nEpsilon = {eps}:")
    print(f"  - Simulaciones promedio: {df_eps['num_simulations'].mean():.0f}")
    print(f"  - Tiempo de mezcla promedio: {df_eps['mixing_time'].mean():.0f}")
    print(f"  - Tiempo ejecución promedio: {df_eps['elapsed_time'].mean():.2f} segundos")

In [ ]:
# Visualización de resultados Hard-Core
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Estimaciones vs tamaño del lattice
for eps in epsilons_hc:
    df_eps = df_hardcore[df_hardcore['epsilon'] == eps]
    axes[0, 0].semilogy(df_eps['K'], df_eps['estimate'], 'o-', label=f'ε={eps}')

axes[0, 0].set_xlabel('Tamaño del Lattice (K)')
axes[0, 0].set_ylabel('Número de Configuraciones (log scale)')
axes[0, 0].set_title('Estimaciones del Modelo Hard-Core')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Tiempo de ejecución
for eps in epsilons_hc:
    df_eps = df_hardcore[df_hardcore['epsilon'] == eps]
    axes[0, 1].plot(df_eps['K'], df_eps['elapsed_time'], 'o-', label=f'ε={eps}')

axes[0, 1].set_xlabel('Tamaño del Lattice (K)')
axes[0, 1].set_ylabel('Tiempo de Ejecución (segundos)')
axes[0, 1].set_title('Tiempo de Ejecución vs Tamaño')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Número de simulaciones requeridas
df_eps01 = df_hardcore[df_hardcore['epsilon'] == 0.1]
axes[1, 0].bar(df_eps01['K'].astype(str), df_eps01['num_simulations'])
axes[1, 0].set_xlabel('Tamaño del Lattice (K)')
axes[1, 0].set_ylabel('Número de Simulaciones')
axes[1, 0].set_title('Simulaciones Requeridas (ε=0.1)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Tiempo de mezcla
axes[1, 1].plot(df_eps01['K'], df_eps01['mixing_time'], 'go-', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Tamaño del Lattice (K)')
axes[1, 1].set_ylabel('Tiempo de Mezcla (pasos)')
axes[1, 1].set_title('Tiempo de Mezcla del Gibbs Sampler (ε=0.1)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Comparación Hard-Core con Valores Exactos (lattices pequeños)

In [ ]:
def compare_hardcore_exact():
    """
    Compara aproximaciones Hard-Core con valores exactos para lattices pequeños.
    """
    K_small = [2, 3, 4]
    epsilon = 0.1
    
    results = []
    
    print("\n=== Comparación Hard-Core con Conteo Exacto ===")
    print("K\tExacto\t\tAproximado\tError Relativo (%)")
    print("-" * 50)
    
    for K in K_small:
        lattice = LatticeGraph(K)
        
        exact_count = exact_hardcore_small(K)
        
        config = MCMCConfig(epsilon=epsilon, num_samples=500)
        mcmc = HardCoreMCMC(lattice, config)
        approx_count, _ = mcmc.count_approximate(verbose=False)
        
        error = abs(approx_count - exact_count) / exact_count * 100
        
        print(f"{K}\t{exact_count:,}\t\t{approx_count:.0f}\t\t{error:.2f}%")
        
        results.append({
            'K': K,
            'exact': exact_count,
            'approximate': approx_count,
            'relative_error': error
        })
    
    return pd.DataFrame(results)

df_hc_comparison = compare_hardcore_exact()

## Resumen y Conclusiones

### Parámetros Utilizados en los Experimentos

1. **q-Coloraciones:**
   - Valores de ε (precisión): 0.5, 0.2, 0.1, 0.05
   - Rango de K: 3 ≤ K ≤ 20
   - Rango de q: 2 ≤ q ≤ 15 (respetando q > 2d*)

2. **Modelo Hard-Core:**
   - Valores de ε (precisión): 0.5, 0.2, 0.1
   - Rango de K: 3 ≤ K ≤ 20

### Observaciones Principales

In [ ]:
# Análisis final y tabla resumen
print("\n" + "="*70)
print("RESUMEN DE EXPERIMENTOS")
print("="*70)

print("\n1. Q-COLORACIONES:")
print("-" * 40)
if not df_q_coloring.empty:
    print(f"Total de experimentos realizados: {len(df_q_coloring)}")
    print(f"Rango de K explorado: {df_q_coloring['K'].min()} - {df_q_coloring['K'].max()}")
    print(f"Rango de q explorado: {df_q_coloring['q'].min()} - {df_q_coloring['q'].max()}")
    print(f"Tiempo total de cómputo: {df_q_coloring['elapsed_time'].sum():.2f} segundos")
    print(f"Tiempo promedio por experimento: {df_q_coloring['elapsed_time'].mean():.3f} segundos")

print("\n2. MODELO HARD-CORE:")
print("-" * 40)
if not df_hardcore.empty:
    print(f"Total de experimentos realizados: {len(df_hardcore)}")
    print(f"Rango de K explorado: {df_hardcore['K'].min()} - {df_hardcore['K'].max()}")
    print(f"Tiempo total de cómputo: {df_hardcore['elapsed_time'].sum():.2f} segundos")
    print(f"Tiempo promedio por experimento: {df_hardcore['elapsed_time'].mean():.3f} segundos")

print("\n3. COMPARACIÓN CON VALORES EXACTOS:")
print("-" * 40)
if not df_comparison.empty:
    print(f"Error relativo promedio (q-coloraciones): {df_comparison['relative_error'].mean():.2f}%")
if not df_hc_comparison.empty:
    print(f"Error relativo promedio (Hard-Core): {df_hc_comparison['relative_error'].mean():.2f}%")

In [ ]:
# Guardar resultados en CSV
output_dir = '../resultados/'
os.makedirs(output_dir, exist_ok=True)

df_q_coloring.to_csv(os.path.join(output_dir, 'q_coloring_results.csv'), index=False)
df_hardcore.to_csv(os.path.join(output_dir, 'hardcore_results.csv'), index=False)
df_comparison.to_csv(os.path.join(output_dir, 'q_coloring_comparison.csv'), index=False)
df_hc_comparison.to_csv(os.path.join(output_dir, 'hardcore_comparison.csv'), index=False)

print("\nResultados guardados en la carpeta 'resultados/'")

## Conclusiones

1. **Eficiencia del Algoritmo**: El algoritmo de conteo aproximado basado en MCMC muestra un comportamiento polinomial en el tamaño del problema, como predice el Teorema 9.1.

2. **Precisión vs Costo Computacional**: Valores más pequeños de ε proporcionan mayor precisión pero requieren significativamente más simulaciones y tiempo de cómputo.

3. **Escalabilidad**: Los métodos implementados escalan razonablemente bien para lattices de tamaño moderado (hasta 20x20).

4. **Validación**: La comparación con valores exactos en lattices pequeños valida la correctitud de nuestra implementación.

### Referencias

- Material del curso: Cadenas de Markov y Aplicaciones
- Teorema 9.1 sobre esquemas de aproximación polinomial aleatorizada
- Técnicas de muestreo de Gibbs para modelos en grafos